In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader
from pathlib import Path
from typing import Optional, Union, Callable

# Constants
batch_size = 32
num_classes = 102
num_epochs = 100
learning_rate = 0.001

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Load the Oxford 102 Flower dataset
data_dir = Path('./data')

train_dataset = datasets.Flowers102(
    root=data_dir,
    split='train',
    transform=data_transforms['train'],
    download=True
)

test_dataset = datasets.Flowers102(
    root=data_dir,
    split='test',
    transform=data_transforms['test'],
    download=True
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Use a pre-trained model from torchvision as a base model (ResNet-18)
model = models.resnet18(pretrained=True)

# Modify the final layer to match the number of classes in the Oxford 102 Flower dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Print loss for the epoch
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

# Testing function
# def test_model(model, test_loader):
#     model.eval()
#     correct = 0
#     total = 0
    
#     with torch.no_grad():
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
    
#     accuracy = correct / total
#     print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Train and evaluate the model
# train_model(model, train_loader, criterion, optimizer, num_epochs)
# test_model(model, test_loader)

ModuleNotFoundError: No module named 'torch'